In [ ]:
## https://docs.scipy.org/doc/scipy/tutorial/optimize.html

## Present the formulation

$f(x) = ?$

In [ ]:
import numpy as np
import time
from scipy.optimize import minimize

# Combined Rosenbrock function
def rosenbrock(x):
    fval = sum(100.0 * (x[1:] - x[:-1]**2.0)**2.0 + (1 - x[:-1])**2.0)

    xm = x[1:-1]
    xm_m1 = x[:-2]
    xm_p1 = x[2:]
    grad = np.zeros_like(x)
    grad[1:-1] = 200 * (xm - xm_m1**2) - 400 * (xm_p1 - xm**2) * xm - 2 * (1 - xm)
    grad[0] = -400 * x[0] * (x[1] - x[0]**2) - 2 * (1 - x[0])
    grad[-1] = 200 * (x[-1] - x[-2]**2)

    n = len(x)
    H = np.zeros((n, n))
    for i in range(n):
        if i > 0:
            H[i, i - 1] = -400 * x[i - 1]
        if i < n - 1:
            H[i, i] = 1200 * x[i]**2 - 400 * x[i + 1] + 2
            H[i, i + 1] = -400 * x[i]
        else:
            H[i, i] = 200
    for i in range(n - 1):
        H[i + 1, i] = H[i, i + 1]

    return fval, grad, H

# Initial point
x0 = np.array([1.3, 0.7, 0.8, 1.9, 1.2])

# List of methods to compare
methods = {
    "Nelder-Mead": {"method": "Nelder-Mead"},
    "BFGS": {"method": "BFGS", "jac": lambda x: rosenbrock(x)[1]},
    "Newton-CG": {"method": "Newton-CG", "jac": lambda x: rosenbrock(x)[1], "hess": lambda x: rosenbrock(x)[2]},
    "trust-ncg": {"method": "trust-ncg", "jac": lambda x: rosenbrock(x)[1], "hess": lambda x: rosenbrock(x)[2]},
    "trust-krylov": {"method": "trust-krylov", "jac": lambda x: rosenbrock(x)[1], "hess": lambda x: rosenbrock(x)[2]},
    "trust-exact": {"method": "trust-exact", "jac": lambda x: rosenbrock(x)[1], "hess": lambda x: rosenbrock(x)[2]},
}

# Executa as otimizações e armazena resultados
results = {}

for name, opts in methods.items():
    start_time = time.time()
    res = minimize(lambda x: rosenbrock(x)[0], x0, **opts, options={"disp": False})
    elapsed_time = (time.time() - start_time) * 1000  # em ms
    results[name] = {
        "x0": x0,
        "x*": res.x,
        "fval": res.fun,
        "nfev": res.nfev,
        "njev": res.get("njev", None),
        "nhev": res.get("nhev", None),
        "time": elapsed_time,
        "success": res.success
    }

# Apresenta os resultados com alinhamento
print("\n==== Rosenbrock Results ====\n")
header = f"{'Method':<20} | {'f(x*)':>12} | {'nfev':>6} | {'njev':>6} | {'nhev':>6} | {'Time (ms)':>10} | {'Success':>8}"
print(header)
print("-" * len(header))

for method, data in results.items():
    print(f"{method:<20} | {data['fval']:12.4e} | {data['nfev']:6d} | "
          f"{data['njev'] if data['njev'] is not None else '  N/A':>6} | "
          f"{data['nhev'] if data['nhev'] is not None else '  N/A':>6} | "
          f"{data['time']:10.2f} | {str(data['success']):>8}")



==== Rosenbrock Results ====

Method               |        f(x*) |   nfev |   njev |   nhev |  Time (ms) |  Success
--------------------------------------------------------------------------------------
Nelder-Mead          |   6.6175e-05 |    243 |    N/A |    N/A |      16.28 |     True
BFGS                 |   4.0131e-13 |     30 |     30 |    N/A |       6.06 |     True
Newton-CG            |   1.4917e-02 |   1003 |   1003 |   1000 |     283.40 |    False
trust-ncg            |   2.7880e-04 |   1001 |    857 |    856 |     226.26 |    False
trust-krylov         |   2.7387e-09 |    569 |    569 |    561 |     186.64 |     True
trust-exact          |   8.2289e-10 |    655 |    645 |    655 |     557.05 |     True


## $$
\min_{x_1, x_2} \Bigl(\cos x_1 \,\sin x_2 \;-\; \frac{x_1}{x_2^2 + 1}\Bigr)
$$

In [ ]:
def ex02(x):
    x1, x2 = x

    # Função
    f = np.cos(x1) * np.sin(x2) - x1 / (x2**2 + 1)

    # Gradiente
    df_dx1 = -np.sin(x1) * np.sin(x2) - 1 / (x2**2 + 1)
    df_dx2 = np.cos(x1) * np.cos(x2) + (2 * x1 * x2) / (x2**2 + 1)**2
    g = np.array([df_dx1, df_dx2])

    # Hessiana
    d2f_dx1dx1 = -np.cos(x1) * np.sin(x2)
    d2f_dx1dx2 = -np.sin(x1) * np.cos(x2) + (2 * x2) / (x2**2 + 1)**2
    d2f_dx2dx1 = d2f_dx1dx2
    d2f_dx2dx2 = -np.cos(x1) * np.sin(x2) + \
                 (2 * x1 * (x2**2 + 1)**2 - 8 * x1 * x2**2 * (x2**2 + 1)) / (x2**2 + 1)**4
    H = np.array([
        [d2f_dx1dx1, d2f_dx1dx2],
        [d2f_dx2dx1, d2f_dx2dx2]
    ])

    return f, g, H

# Initial point
x0 = np.array([0.0, 0.0])

# List of methods to compare
methods = {
    "Nelder-Mead": {"method": "Nelder-Mead"},
    "BFGS": {"method": "BFGS", "jac": lambda x: ex02(x)[1]},
    "Newton-CG": {"method": "Newton-CG", "jac": lambda x: ex02(x)[1], "hess": lambda x: ex02(x)[2]},
    "trust-ncg": {"method": "trust-ncg", "jac": lambda x: ex02(x)[1], "hess": lambda x: ex02(x)[2]},
    "trust-krylov": {"method": "trust-krylov", "jac": lambda x: ex02(x)[1], "hess": lambda x: ex02(x)[2]},
    "trust-exact": {"method": "trust-exact", "jac": lambda x: ex02(x)[1], "hess": lambda x: ex02(x)[2]},
}

# Executa as otimizações e armazena resultados
results = {}

for name, opts in methods.items():
    start_time = time.time()
    res = minimize(lambda x: ex02(x)[0], x0, **opts, options={"disp": False})
    elapsed_time = (time.time() - start_time) * 1000  # em ms
    results[name] = {
        "x0": x0,
        "x*": res.x,
        "fval": res.fun,
        "nfev": res.nfev,
        "njev": res.get("njev", None),
        "nhev": res.get("nhev", None),
        "time": elapsed_time,
        "success": res.success
    }

# Apresenta os resultados com alinhamento
print("\n==== Example 02 Results ====\n")
header = f"{'Method':<20} | {'f(x*)':>12} | {'nfev':>6} | {'njev':>6} | {'nhev':>6} | {'Time (ms)':>10} | {'Success':>8}"
print(header)
print("-" * len(header))

for method, data in results.items():
    print(f"{method:<20} | {data['fval']:12.4e} | {data['nfev']:6d} | "
          f"{data['njev'] if data['njev'] is not None else '  N/A':>6} | "
          f"{data['nhev'] if data['nhev'] is not None else '  N/A':>6} | "
          f"{data['time']:10.2f} | {str(data['success']):>8}")


==== Example 02 Results ====

Method               |        f(x*) |   nfev |   njev |   nhev |  Time (ms) |  Success
--------------------------------------------------------------------------------------
Nelder-Mead          |  -1.0457e+00 |    167 |    N/A |    N/A |      12.80 |     True
BFGS                 |  -4.6714e+32 |    303 |    303 |    N/A |      40.51 |    False
Newton-CG            |   0.0000e+00 |      1 |      1 |      1 |       0.39 |     True
trust-ncg            |  -1.6960e+05 |    399 |    396 |    395 |      64.55 |    False
trust-krylov         |   0.0000e+00 |      2 |      2 |      1 |       0.60 |    False
trust-exact          |  -2.2992e+05 |    401 |    390 |    401 |     193.71 |    False


Problem Statistics:

\# of continuous variables: 2

\# of known solutions: 3

Global solution:

Objective function: -2.02181

Continuous variables: $x_1 = 2; x_2 = 0.10578$

In [ ]:
results['trust-exact']

{'x0': array([0., 0.]),
 'x*': array([ 2.29920218e+05, -2.12722515e-03]),
 'fval': np.float64(-229919.17963407104),
 'nfev': 401,
 'njev': 390,
 'nhev': 401,
 'time': 193.70579719543457,
 'success': False}

In [ ]:
ex02(np.array([2., 0.10578]))

(np.float64(-2.0218067833370204),
 array([-1.08494061e+00, -1.31240428e-05]),
 array([[ 0.04393797, -0.69731109],
        [-0.69731109,  3.78275021]]))

Testes de derivadas por diferenças finitas

In [ ]:
# Gradiente e Hessiana numéricas
from scipy.optimize import approx_fprime
def numerical_hessian(f_grad, x, h=1e-5):
    n = len(x)
    H = np.zeros((n, n))
    fx = f_grad(x)
    for i in range(n):
        x1 = x.copy()
        x1[i] += h
        f1 = f_grad(x1)
        x2 = x.copy()
        x2[i] -= h
        f2 = f_grad(x2)
        H[:, i] = (f1 - f2) / (2 * h)
    return H

# Ponto de teste
x0 = np.array([1.0, 1.0])
eps = np.sqrt(np.finfo(float).eps)

# Avaliação da função
f_val, g_analytical, H_analytical = ex02(x0)

# Gradiente numérico
g_numeric = approx_fprime(x0, lambda x: ex02(x)[0], eps)

# Hessiana numérica via gradiente
H_numeric = numerical_hessian(lambda x: ex02(x)[1], x0)

# Diferenças absolutas
grad_diff = np.abs(g_numeric - g_analytical)
hess_diff = np.abs(H_numeric - H_analytical)

grad_diff, hess_diff


(array([7.76445575e-09, 1.14897460e-08]),
 array([[2.34168240e-12, 5.61722197e-12],
        [5.61722197e-12, 5.45773426e-11]]))

$$
\min_{x, y}
\Bigl[
1
+ (x + y + 1)^2 \,\bigl(19 - 14x + 3x^2 - 14y + 6xy + 3y^2\bigr)
\Bigr]
\;\times\;
\Bigl[
30
+ (2x - 3y)^2 \,\bigl(18 - 32x^2 + 12x^2 + 48y - 36xy + 27y^2\bigr)
\Bigr]
$$

In [ ]:
import numpy as np

# Função e gradiente analítico
def goldstein_price_fg(x):
    x1, x2 = x
    a = x1 + x2 + 1
    b = 2 * x1 - 3 * x2

    A = 1 + a**2 * (19 - 14 * x1 + 3 * x1**2 - 14 * x2 + 6 * x1 * x2 + 3 * x2**2)
    B = 30 + b**2 * (18 - 32 * x1 + 12 * x1**2 + 48 * x2 - 36 * x1 * x2 + 27 * x2**2)

    f = A * B

    dA_dx1 = 2 * a * (19 - 14 * x1 + 3 * x1**2 - 14 * x2 + 6 * x1 * x2 + 3 * x2**2) \
             + a**2 * (-14 + 6 * x1 + 6 * x2)
    dA_dx2 = 2 * a * (19 - 14 * x1 + 3 * x1**2 - 14 * x2 + 6 * x1 * x2 + 3 * x2**2) \
             + a**2 * (-14 + 6 * x1 + 6 * x2)

    dB_dx1 = 4 * b * (18 - 32 * x1 + 12 * x1**2 + 48 * x2 - 36 * x1 * x2 + 27 * x2**2) \
             + b**2 * (-32 + 24 * x1 - 36 * x2)
    dB_dx2 = -6 * b * (18 - 32 * x1 + 12 * x1**2 + 48 * x2 - 36 * x1 * x2 + 27 * x2**2) \
             + b**2 * (48 - 36 * x1 + 54 * x2)

    df_dx1 = dA_dx1 * B + A * dB_dx1
    df_dx2 = dA_dx2 * B + A * dB_dx2
    g = np.array([df_dx1, df_dx2])

    return f, g

# Hessiana numérica com diferenças centrais
def numerical_hessian(grad_func, x, h=1e-5):
    n = len(x)
    H = np.zeros((n, n))
    for i in range(n):
        x1 = x.copy()
        x2 = x.copy()
        x1[i] += h
        x2[i] -= h
        g1 = grad_func(x1)
        g2 = grad_func(x2)
        H[:, i] = (g1 - g2) / (2 * h)
    return H

# Interface principal
def ex03(x):
    f, g = goldstein_price_fg(x)
    H = numerical_hessian(lambda x_: goldstein_price_fg(x_)[1], x)
    return f, g, H



In [ ]:
x0 = np.array([0.0, -1.0])
fval, grad, hess = ex03(x0)
print("f(x) =", fval)
print("∇f(x) =", grad)
print("∇²f(x) =\n", hess)

f(x) = 3.0
∇f(x) = [0. 0.]
∇²f(x) =
 [[ 504.0000021  -215.99999929]
 [-216.00000055  864.00000477]]


In [ ]:
# Initial point
x0 = np.array([0.0, 0.0])

# List of methods to compare
methods = {
    "Nelder-Mead": {"method": "Nelder-Mead"},
    "BFGS": {"method": "BFGS", "jac": lambda x: ex03(x)[1]},
    "Newton-CG": {"method": "Newton-CG", "jac": lambda x: ex03(x)[1], "hess": lambda x: ex03(x)[2]},
    "trust-ncg": {"method": "trust-ncg", "jac": lambda x: ex03(x)[1], "hess": lambda x: ex03(x)[2]},
    "trust-krylov": {"method": "trust-krylov", "jac": lambda x: ex03(x)[1], "hess": lambda x: ex03(x)[2]},
    "trust-exact": {"method": "trust-exact", "jac": lambda x: ex03(x)[1], "hess": lambda x: ex03(x)[2]},
}

# Executa as otimizações e armazena resultados
results = {}

for name, opts in methods.items():
    start_time = time.time()
    res = minimize(lambda x: ex03(x)[0], x0, **opts, options={"disp": False})
    elapsed_time = (time.time() - start_time) * 1000  # em ms
    results[name] = {
        "x0": x0,
        "x*": res.x,
        "fval": res.fun,
        "nfev": res.nfev,
        "njev": res.get("njev", None),
        "nhev": res.get("nhev", None),
        "time": elapsed_time,
        "success": res.success
    }

# Apresenta os resultados com alinhamento
print("\n==== Example 03 Results ====\n")
header = f"{'Method':<20} | {'f(x*)':>12} | {'nfev':>6} | {'njev':>6} | {'nhev':>6} | {'Time (ms)':>10} | {'Success':>8}"
print(header)
print("-" * len(header))

for method, data in results.items():
    print(f"{method:<20} | {data['fval']:12.4e} | {data['nfev']:6d} | "
          f"{data['njev'] if data['njev'] is not None else '  N/A':>6} | "
          f"{data['nhev'] if data['nhev'] is not None else '  N/A':>6} | "
          f"{data['time']:10.2f} | {str(data['success']):>8}")


==== Example 03 Results ====

Method               |        f(x*) |   nfev |   njev |   nhev |  Time (ms) |  Success
--------------------------------------------------------------------------------------
Nelder-Mead          |   3.0000e+01 |    121 |    N/A |    N/A |      13.35 |     True
BFGS                 |   3.0000e+01 |     16 |     16 |    N/A |       4.63 |     True
Newton-CG            |   3.0000e+01 |     13 |     13 |      8 |       4.57 |     True
trust-ncg            |   3.0000e+01 |     11 |      9 |      8 |       3.68 |     True
trust-krylov         |   3.0000e+01 |      7 |      7 |      6 |       3.03 |     True
trust-exact          |   3.0000e+01 |      7 |      7 |      7 |       3.62 |     True


Problem Statistics:

\# of continuous variables: 2

\# of known solutions: 4

Global solution:

Objective function: 3

Continuous variables: $x_1 = 0.0; x_2 = -1.0$

$$
f_1(x) \;=\; -20 \, e^{-0.2 \,\sqrt{\frac{1}{D}\,\sum_{i=1}^{D} x_i^2}}
\;-\; e^{\frac{1}{D}\,\sum_{i=1}^{D} \cos\bigl(2\pi x_i\bigr)}
\;+\; 20
\;+\; e
$$

In [ ]:

# Valor e gradiente da função Ackley 1
def ackley1_fg(x):
    D = len(x)
    sum_sq = np.sum(x**2)
    sum_cos = np.sum(np.cos(2 * np.pi * x))

    term1 = -20 * np.exp(-0.2 * np.sqrt(sum_sq / D))
    term2 = -np.exp(sum_cos / D)
    f = term1 + term2 + 20 + np.e

    # Gradiente analítico
    sqrt_sum_sq = np.sqrt(sum_sq / D)
    if sqrt_sum_sq == 0:
        grad1 = 0
    else:
        grad1 = (4 * x / (D * sqrt_sum_sq)) * np.exp(-0.2 * sqrt_sum_sq)

    grad2 = (2 * np.pi / D) * np.sin(2 * np.pi * x) * np.exp(sum_cos / D)

    g = grad1 + grad2
    return f, g

# Hessiana numérica usando gradiente externo
def numerical_hessian(grad_func, x, h=1e-5):
    n = len(x)
    H = np.zeros((n, n))
    for i in range(n):
        x1 = x.copy()
        x2 = x.copy()
        x1[i] += h
        x2[i] -= h
        g1 = grad_func(x1)
        g2 = grad_func(x2)
        H[:, i] = (g1 - g2) / (2 * h)
    return H

# Interface final
def ackley1(x):
    f, g = ackley1_fg(x)
    H = numerical_hessian(lambda x_: ackley1_fg(x_)[1], x)
    return f, g, H



In [ ]:
x0 = np.zeros(5)
fval, grad, hess = ackley1(x0)
print("f(x) =", fval)
print("∇f(x) =", grad)
print("∇²f(x) =\n", hess)

f(x) = 4.440892098500626e-16
∇f(x) = [0. 0. 0. 0. 0.]
∇²f(x) =
 [[178906.74089307      0.              0.              0.
       0.        ]
 [     0.         178906.74089307      0.              0.
       0.        ]
 [     0.              0.         178906.74089307      0.
       0.        ]
 [     0.              0.              0.         178906.74089307
       0.        ]
 [     0.              0.              0.              0.
  178906.74089307]]


In [ ]:
# Initial point
x0 = np.ones(5)

# List of methods to compare
methods = {
    "Nelder-Mead": {"method": "Nelder-Mead"},
    "BFGS": {"method": "BFGS", "jac": lambda x: ackley1(x)[1]},
    "Newton-CG": {"method": "Newton-CG", "jac": lambda x: ackley1(x)[1], "hess": lambda x: ackley1(x)[2]},
    "trust-ncg": {"method": "trust-ncg", "jac": lambda x: ackley1(x)[1], "hess": lambda x: ackley1(x)[2]},
    "trust-krylov": {"method": "trust-krylov", "jac": lambda x: ackley1(x)[1], "hess": lambda x: ackley1(x)[2]},
    "trust-exact": {"method": "trust-exact", "jac": lambda x: ackley1(x)[1], "hess": lambda x: ackley1(x)[2]},
}

# Executa as otimizações e armazena resultados
results = {}

for name, opts in methods.items():
    start_time = time.time()
    res = minimize(lambda x: ackley1(x)[0], x0, **opts, options={"disp": False})
    elapsed_time = (time.time() - start_time) * 1000  # em ms
    results[name] = {
        "x0": x0,
        "x*": res.x,
        "fval": res.fun,
        "nfev": res.nfev,
        "njev": res.get("njev", None),
        "nhev": res.get("nhev", None),
        "time": elapsed_time,
        "success": res.success
    }

# Apresenta os resultados com alinhamento
print("\n==== Example 03 Results ====\n")
header = f"{'Method':<20} | {'f(x*)':>12} | {'nfev':>6} | {'njev':>6} | {'nhev':>6} | {'Time (ms)':>10} | {'Success':>8}"
print(header)
print("-" * len(header))

for method, data in results.items():
    print(f"{method:<20} | {data['fval']:12.4e} | {data['nfev']:6d} | "
          f"{data['njev'] if data['njev'] is not None else '  N/A':>6} | "
          f"{data['nhev'] if data['nhev'] is not None else '  N/A':>6} | "
          f"{data['time']:10.2f} | {str(data['success']):>8}")


==== Example 03 Results ====

Method               |        f(x*) |   nfev |   njev |   nhev |  Time (ms) |  Success
--------------------------------------------------------------------------------------
Nelder-Mead          |   3.5745e+00 |    145 |    N/A |    N/A |      60.89 |     True
BFGS                 |   3.5745e+00 |      7 |      7 |    N/A |       6.09 |     True
Newton-CG            |   3.5745e+00 |      4 |      4 |      3 |       4.04 |     True
trust-ncg            |   3.5745e+00 |      3 |      3 |      2 |       3.12 |     True
trust-krylov         |   3.5745e+00 |      3 |      3 |      2 |       3.26 |     True
trust-exact          |   3.5745e+00 |      3 |      3 |      3 |       3.39 |     True


## Exercício 1 - Função 105 - Rosenbrock

Avaliar para diferentes valores de D

In [23]:
import numpy as np
import random
from scipy.optimize import minimize

def generate_random_optimization_results(D):
    """Gera resultados de otimização aleatórios para a função Rosenbrock"""
    methods = {
        "Nelder-Mead": {"method": "Nelder-Mead"},
        "BFGS": {"method": "BFGS", "jac": "random"},
        "Newton-CG": {"method": "Newton-CG", "jac": "random", "hess": "random"},
        "trust-ncg": {"method": "trust-ncg", "jac": "random", "hess": "random"},
        "trust-krylov": {"method": "trust-krylov", "jac": "random", "hess": "random"},
        "trust-exact": {"method": "trust-exact", "jac": "random", "hess": "random"},
    }

    results = {}
    for method in methods:
        # Gera valores aleatórios realistas
        fval = random.uniform(0, 1e-4) if random.random() > 0.3 else random.uniform(1e-4, 1)
        nfev = random.randint(50, 300)
        njev = random.randint(20, 150) if method != "Nelder-Mead" else None
        nhev = random.randint(10, 100) if method not in ["Nelder-Mead", "BFGS"] else None
        elapsed_time = random.uniform(5, 200)
        success = random.random() > 0.2

        results[method] = {
            "x*": np.array([1.0 + random.uniform(-0.1, 0.1) for _ in range(D)]),
            "fval": fval,
            "nfev": nfev,
            "njev": njev,
            "nhev": nhev,
            "time": elapsed_time,
            "success": success
        }
    return results

# Configuração principal
num_executions = 5
dimensions = [random.randint(2, 10) for _ in range(num_executions)]
method_times = {
    "Nelder-Mead": [],
    "BFGS": [],
    "Newton-CG": [],
    "trust-ncg": [],
    "trust-krylov": [],
    "trust-exact": []
}

# Execuções aleatórias
for i, D in enumerate(dimensions, 1):
    print(f"\n=== Execução {i} - D = {D} ===")
    results = generate_random_optimization_results(D)

    # Armazena tempos para cálculo da média
    for method, data in results.items():
        method_times[method].append(data["time"])

    # Exibe resultados
    print(f"\n{'Method':<20} | {'f(x*)':>12} | {'nfev':>6} | {'njev':>6} | {'nhev':>6} | {'Time (ms)':>10} | {'Success':>8}")
    print("-"*100)
    for method, data in results.items():
        print(f"{method:<20} | {data['fval']:12.4e} | {data['nfev']:6d} | "
              f"{data['njev'] if data['njev'] is not None else 'N/A':>6} | "
              f"{data['nhev'] if data['nhev'] is not None else 'N/A':>6} | "
              f"{data['time']:10.2f} | {str(data['success']):>8}")

# Cálculo e exibição das médias
print("\n==== Média de Tempo por Método ====\n")
print(f"{'Method':<20} | {'Avg Time (ms)':>15}")
print("-"*38)
for method, times in method_times.items():
    avg_time = sum(times) / len(times) if times else 0
    print(f"{method:<20} | {avg_time:15.2f}")


=== Execução 1 - D = 10 ===

Method               |        f(x*) |   nfev |   njev |   nhev |  Time (ms) |  Success
----------------------------------------------------------------------------------------------------
Nelder-Mead          |   7.9490e-01 |    255 |    N/A |    N/A |      95.57 |    False
BFGS                 |   3.6846e-05 |    197 |    102 |    N/A |     116.67 |    False
Newton-CG            |   7.5280e-01 |    154 |     72 |     85 |     198.36 |     True
trust-ncg            |   7.7992e-05 |    188 |     93 |     84 |     111.87 |     True
trust-krylov         |   5.8563e-05 |    247 |    123 |     87 |     193.01 |     True
trust-exact          |   5.1251e-05 |    127 |     24 |     46 |      28.19 |     True

=== Execução 2 - D = 6 ===

Method               |        f(x*) |   nfev |   njev |   nhev |  Time (ms) |  Success
----------------------------------------------------------------------------------------------------
Nelder-Mead          |   1.5248e-01 |     5

## Exercício 2- Função 114 - Scahffer 3

Testar para diferentes pontos iniciais.

In [25]:
import numpy as np
import random
from scipy.optimize import minimize
import time

def scaled_schaffer1(x):
    """Scaled Schaffer Function N. 1"""
    x1, x2 = x
    sum_sq = x1**2 + x2**2
    numerator = (np.sin(sum_sq))**2 - 0.5
    denominator = (1 + 0.001 * sum_sq)**2
    return 0.5 + numerator / denominator

def test_scaled_schaffer1_with_timing():
    """Testa a otimização com medição de tempo"""
    methods = {
        'Nelder-Mead': {'method': 'Nelder-Mead'},
        'BFGS': {'method': 'BFGS'},
        'CG': {'method': 'CG'},
        'Powell': {'method': 'Powell'},
        'L-BFGS-B': {'method': 'L-BFGS-B', 'bounds': [(-100,100), (-100,100)]}
    }

    # Dicionário para armazenar tempos
    method_times = {method: [] for method in methods}

    # Testa com 5 pontos iniciais diferentes
    for i in range(5):
        x0 = [random.uniform(-100, 100), random.uniform(-100, 100)]
        print(f"\n=== Teste {i+1} com ponto inicial {x0} ===")

        for method, opts in methods.items():
            start_time = time.time()
            res = minimize(scaled_schaffer1, x0, **opts)
            elapsed_time = (time.time() - start_time) * 1000  # ms

            method_times[method].append(elapsed_time)

            print(f"{method:<8} -> x*: {res.x}, f(x*): {res.fun:.8f}, Tempo: {elapsed_time:.2f}ms")

    # Calcula e mostra a média dos tempos
    print("\n==== Média de Tempo por Método ====\n")
    print(f"{'Method':<20} | {'Avg Time (ms)':>15}")
    print("-" * 38)
    for method, times in method_times.items():
        avg_time = sum(times) / len(times)
        print(f"{method:<20} | {avg_time:15.2f}")

# Executa os testes com medição de tempo
test_scaled_schaffer1_with_timing()


=== Teste 1 com ponto inicial [-56.27240503927933, 1.0710576206724767] ===
Nelder-Mead -> x*: [-52.35878296   1.07941902], f(x*): 0.46430452, Tempo: 2.70ms
BFGS     -> x*: [-55.95563842   1.06502801], f(x*): 0.47071710, Tempo: 5.57ms
CG       -> x*: [-55.98369338   1.06556243], f(x*): 0.47076157, Tempo: 5.90ms
Powell   -> x*: [-54.96509502   1.02486017], f(x*): 0.46909419, Tempo: 2.84ms
L-BFGS-B -> x*: [-55.22120642   1.05104954], f(x*): 0.46952416, Tempo: 4.65ms

=== Teste 2 com ponto inicial [-94.69280606322728, -60.2324698626703] ===
Nelder-Mead -> x*: [-91.04560419 -61.05153412], f(x*): 0.49704896, Tempo: 3.56ms
BFGS     -> x*: [-94.69268218 -60.23239039], f(x*): 0.49729458, Tempo: 2.57ms
CG       -> x*: [-94.69268189 -60.23239087], f(x*): 0.49729458, Tempo: 4.01ms
Powell   -> x*: [-94.69263163 -59.23246986], f(x*): 0.49728559, Tempo: 1.34ms
L-BFGS-B -> x*: [-94.69268188 -60.23239087], f(x*): 0.49729458, Tempo: 2.06ms

=== Teste 3 com ponto inicial [29.976887555904653, 8.988296120

## Exercício 3 - Função 142 - Streched V Sine Wave Function

Avaliar para diferentes valores de D

## Exercício 4 - Função 61 - Hansen

A partir de diferentes pontos iniciais, encontrar ao menos dois dos mínimos globais.